# `DataFrame` Querying

In the previous chapter we discussed accessing rows of a `DataFrame` by row position.  In practice, I don't find this particularly useful. 
 Rather, I find that I usually access rows by some kind of logical condition, which is referred to as *querying* the `DataFrame`.
 
In this chapter we discuss two ways of querying a `DataFrame`:

1. masking
2. the `DataFrame.query()` method.

## Importing Packages

Let's first import the packages that we will need.

In [1]:
import pandas as pd
import yfinance as yf
pd.set_option('display.max_rows', 10)

## Reading-In Data

Next, let's use `pandas_datareader` to read-in some `SPY` data from July 2021.

In [2]:
df_spy = yf.download('SPY', start='2021-06-30', end='2021-07-31', auto_adjust=False, rounding=True)
df_spy.head()

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,
2021-06-30,404.51,428.06,428.78,427.18,427.21,64827900
2021-07-01,406.75,430.43,430.60,428.80,428.87,53441000
2021-07-02,409.86,433.72,434.10,430.52,431.67,57697700
2021-07-06,409.11,432.93,434.01,430.01,433.78,68710400
2021-07-07,410.56,434.46,434.76,431.51,433.66,63549500


The following code:

- removes the `SPY` level of the column index
- resets the index so that `Date` is a regular column
- puts the column names into snake-case.

In [3]:
df_spy = df_spy.droplevel(level=1, axis=1)
df_spy = df_spy.rename_axis(None, axis=1)
df_spy.reset_index(inplace=True)
df_spy.columns = df_spy.columns.str.lower().str.replace(' ', '_')
df_spy.head()

,date,adj_close,close,high,low,open,volume
0,2021-06-30,404.51,428.06,428.78,427.18,427.21,64827900
1,2021-07-01,406.75,430.43,430.60,428.80,428.87,53441000
2,2021-07-02,409.86,433.72,434.10,430.52,431.67,57697700
3,2021-07-06,409.11,432.93,434.01,430.01,433.78,68710400
4,2021-07-07,410.56,434.46,434.76,431.51,433.66,63549500


## Comparison and `DataFrame` Columns

As discussed in a previous chapter, a column of a `DataFrame` is a `Series` object, which is a souped up `numpy.array` (think vector or matrix).

Let's separate out the `close` column of `df_spy` and assign it to a variable.

In [4]:
pd.options.display.max_rows = 6 # this modifies the printing of dataframes
ser_close = df_spy['close']
ser_close

0     428.06
1     430.43
2     433.72
       ...  
19    438.83
20    440.65
21    438.51
Name: close, Length: 22, dtype: float64

Recall that a `pandas.Series` is smart with respect to component-wise arithmetic operations, meaning it behaves like a vector from linear algebra.  This means that arithmetic operations are *broadcasted* as you might expect.

For example, division by 100 is broadcasted component-wise.

In [5]:
ser_close / 100

0     4.2806
1     4.3043
2     4.3372
       ...  
19    4.3883
20    4.4065
21    4.3851
Name: close, Length: 22, dtype: float64

#### It is a convenient fact that this broadcasting behavior also occurs with comparison, and produces a `Series` of booleans. 

The following code checks which elements of `ser_adjusted` are greater than 435. 

In [6]:
ser_test = (ser_close > 435)
ser_test

0     False
1     False
2     False
      ...  
19     True
20     True
21     True
Name: close, Length: 22, dtype: bool

Let's check that the resulting variable `ser_test` is a `pandas.Series`.

In [7]:
type(ser_test)

pandas.core.series.Series

And finally let's observe the `.values` elements of `ser_test`.

In [8]:
print(ser_test.values)

[False False False False False False  True  True  True  True False False
 False False False  True  True  True  True  True  True  True]


A few observation about what just happened:

1. When we compare a `Series` of numerical values (`ser_close`) to a single number (`435`), we get back a `Series` of booleans (`ser_test`).

2. We have that `ser_test[i]` = (`ser_adjusted[i] > 435`).

3. So the comparison operation was broadcasted as advertised.

This is easy to see by appending `ser_test` to `df_spy` and then reprinting.

In [9]:
pd.options.display.max_rows = 25
df_spy['test'] = ser_test
df_spy

,date,adj_close,close,high,low,open,volume,test
0,2021-06-30,404.51,428.06,428.78,427.18,427.21,64827900,False
1,2021-07-01,406.75,430.43,430.60,428.80,428.87,53441000,False
2,2021-07-02,409.86,433.72,434.10,430.52,431.67,57697700,False
3,2021-07-06,409.11,432.93,434.01,430.01,433.78,68710400,False
4,2021-07-07,410.56,434.46,434.76,431.51,433.66,63549500,False
5,2021-07-08,407.21,430.92,431.73,427.52,428.78,97595200,False
6,2021-07-09,411.56,435.52,435.84,430.71,432.53,76238600,True
7,2021-07-12,413.03,437.08,437.35,434.97,435.43,52889600,True
8,2021-07-13,411.63,435.59,437.84,435.31,436.24,52911300,True
9,2021-07-14,412.24,436.24,437.92,434.91,437.40,64130400,True


As we will see in the next two sections, the broadcasting of comparison can be used to query subsets of rows of a `DataFrame`.

## `DataFrame` Masking

From the code below we know that `df_spy` has 22 rows.

In [10]:
df_spy.shape

(22, 8)

The following code creates a list consisting of 22 booleans, all of them `False`.

In [11]:
lst_bool = [False] * 22
lst_bool

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

Now, let's see what happens when we feed this `list` of `False` booleans into `df_spy` using square brackets.

In [12]:
df_spy[lst_bool]

,date,adj_close,close,high,low,open,volume,test


---

**Code Challenge:** Verify that `df_spy[lst_bool]` is an empty `DataFrame`.

In [13]:
#| code-fold: true
#| code-summary: "Solution"
type(df_spy[lst_bool])

pandas.core.frame.DataFrame

In [14]:
#| code-fold: true
#| code-summary: "Solution"
df_spy[lst_bool].shape

(0, 8)

--- 

Next, let's modify `lst_bool` slightly by changing the 0th entry to `True`. Then lets feed `lst_bool` into `df_spy` again.

In [15]:
lst_bool[0] = True
df_spy[lst_bool]

,date,adj_close,close,high,low,open,volume,test
0,2021-06-30,404.51,428.06,428.78,427.18,427.21,64827900,False


So what happened?  Notice that `df_spy[lst_bool]` returns a `DataFrame` consisting only of the 0th row of `df_spy`.

Let's modify `lst_bool` once again, by setting the 1st entry of `df_spy` to `True`, and then once again feed it into `df_spy`. 

In [16]:
lst_bool[1] = True
df_spy[lst_bool]

,date,adj_close,close,high,low,open,volume,test
0,2021-06-30,404.51,428.06,428.78,427.18,427.21,64827900,False
1,2021-07-01,406.75,430.43,430.60,428.80,428.87,53441000,False


**Punchline:** What is returned by the code `df_spy[lst_bool]` will be a `DataFrame` consisting of all the rows corresponding to the `True` entries of `lst_bool`.

This is called `DataFrame` *masking*.

--- 

**Code Challenge:** Modify `lst_bool` and then use `DataFrame` masking to grab the 0th, 1st and, 3rd rows of `df_spy`.

In [17]:
#| code-fold: true
#| code-summary: "Solution"
lst_bool[3] = True
df_spy[lst_bool]

,date,adj_close,close,high,low,open,volume,test
0,2021-06-30,404.51,428.06,428.78,427.18,427.21,64827900,False
1,2021-07-01,406.75,430.43,430.60,428.80,428.87,53441000,False
3,2021-07-06,409.11,432.93,434.01,430.01,433.78,68710400,False


---

## Querying with `DataFrame` Masking

We often want to query a `DataFrame` based on some kind of comparison involving its column values.

We can achieve this kind of querying by combining the broadcasting of comparison over `DataFrame` columns with `DataFrame` masking.

In order to consider concrete examples, let's read-in some data.  

The following code reads in a data set consisting of end-of-day prices for four different ETFs (SPY, IWM, QQQ, DIA), during the month of July 2021.

In [18]:
pd.options.display.max_rows = 25
df_etf = yf.download(
    ['SPY', 'QQQ', 'IWM', 'DIA'], start='2021-06-30', end='2021-07-31',
    auto_adjust=False, rounding=True
)
df_etf.head()

[*********************100%***********************]  4 of 4 completed


Price      Adj Close                           Close                           
Ticker           DIA     IWM     QQQ     SPY     DIA     IWM     QQQ     SPY   
Date                                                                           
2021-06-30    320.48  217.77  345.63  404.51  344.95  229.37  354.43  428.06  \
2021-07-01    321.79  219.69  345.76  406.75  346.36  231.39  354.57  430.43   
2021-07-02    323.26  217.60  349.73  409.86  347.94  229.19  358.64  433.72   
2021-07-06    321.29  214.44  351.24  409.11  345.82  225.86  360.19  432.93   
2021-07-07    322.31  212.44  351.99  410.56  346.92  223.76  360.95  434.46   

Price         High          ...     Low            Open                   
Ticker         DIA     IWM  ...     QQQ     SPY     DIA     IWM     QQQ   
Date                        ...                                           
2021-06-30  345.51  230.32  ...  353.83  427.18  342.38  228.65  354.83  \
2021-07-01  346.40  231.85  ...  352.68  428.80  345.78  230.81  354.07   
2021-07-02  348.29  232.08  ...  356.28  430.52  347.04  232.00  356.52   
2021-07-06  348.11  229.46  ...  356.49  430.01  347.75  229.36  359.26   
2021-07-07  347.14  226.67  ...  358.94  431.51  345.65  225.54  362.45   

Price                Volume                                
Ticker         SPY      DIA       IWM       QQQ       SPY  
Date                                                       
2021-06-30  427.21  3778900  26039000  32724000  64827900  
2021-07-01  428.87  3606900  18089100  29290000  53441000  
2021-07-02  431.67  3013500  21029700  32727200  57697700  
2021-07-06  433.78  3910600  27771300  38842400  68710400  
2021-07-07  433.66  3347000  28521500  35265200  63549500  

[5 rows x 24 columns]

This data is not as tidy as we would like.  Let's use method chaining to perform a series of data munging operations.

In [19]:
df_etf = (
    df_etf[[('Close','DIA'), ('Close', 'IWM'), ('Close', 'QQQ'), ('Close', 'SPY')]] # grab close prices
    .stack() # pivot the table
    .reset_index()
    .rename_axis(None, axis=1) # remove the name from the row index
    .rename(columns={'Date':'date', 'Ticker':'symbol', 'Close':'close'}) # columns in snake-case
    .sort_values(by=['symbol', 'date'])
)
df_etf

,date,symbol,close
0,2021-06-30,DIA,344.95
4,2021-07-01,DIA,346.36
8,2021-07-02,DIA,347.94
12,2021-07-06,DIA,345.82
16,2021-07-07,DIA,346.92
...,...,...,...
71,2021-07-26,SPY,441.02
75,2021-07-27,SPY,439.01
79,2021-07-28,SPY,438.83
83,2021-07-29,SPY,440.65


### Querying for One Symbol

We are now ready to apply `DataFrame` masking to our ETF data set.

As a first example, let's isolate all the rows of `df_etf` that correspond to `IWM`.

In [20]:
pd.options.display.max_rows = 6
ser_bool = (df_etf['symbol'] == "IWM")
df_etf[ser_bool]

,date,symbol,close
1,2021-06-30,IWM,229.37
5,2021-07-01,IWM,231.39
9,2021-07-02,IWM,229.19
...,...,...,...
77,2021-07-28,IWM,220.82
81,2021-07-29,IWM,222.52
85,2021-07-30,IWM,221.05


Notice that we did this in two steps: 

1. Calculate the series of `booleans` called `ser_bool` using comparison broadcasting.

2. Perform the masking by using square brackets `[]` and `ser_bool`.

We can actually perform this masking in a single line of code, without creating the intermediate variable `ser_bool`.

In [21]:
df_etf[df_etf['symbol'] == "IWM"]

,date,symbol,close
1,2021-06-30,IWM,229.37
5,2021-07-01,IWM,231.39
9,2021-07-02,IWM,229.19
...,...,...,...
77,2021-07-28,IWM,220.82
81,2021-07-29,IWM,222.52
85,2021-07-30,IWM,221.05


---

**Code Challenge:** Select all the rows of `df_etf` for `QQQ`. 

In [22]:
#| code-fold: true
#| code-summary: "Solution"
df_etf[df_etf['symbol'] == 'QQQ']

,date,symbol,close
2,2021-06-30,QQQ,354.43
6,2021-07-01,QQQ,354.57
10,2021-07-02,QQQ,358.64
...,...,...,...
78,2021-07-28,QQQ,365.83
82,2021-07-29,QQQ,366.48
86,2021-07-30,QQQ,364.57


--- 

### Querying for Multiple Symbols

We can use the `.isin()` method to query a `DataFrame` for multiple symbols.  The technique is to feed `.isin()` a `list` of symbols you want to query for.

The following code grabs all the rows of `df_etf` for both `QQQ` and `DIA`.

In [23]:
df_etf[df_etf['symbol'].isin(['QQQ', 'DIA'])]

,date,symbol,close
0,2021-06-30,DIA,344.95
4,2021-07-01,DIA,346.36
8,2021-07-02,DIA,347.94
...,...,...,...
78,2021-07-28,QQQ,365.83
82,2021-07-29,QQQ,366.48
86,2021-07-30,QQQ,364.57


---

**Code Challenge:** Grab all rows of `df_etf` corresponding to `SPY`, `IWM`, and `QQQ`.

In [24]:
#| code-fold: true
#| code-summary: "Solution"
df_etf[df_etf['symbol'].isin(['SPY', 'IWM', 'QQQ'])]

,date,symbol,close
1,2021-06-30,IWM,229.37
5,2021-07-01,IWM,231.39
9,2021-07-02,IWM,229.19
...,...,...,...
79,2021-07-28,SPY,438.83
83,2021-07-29,SPY,440.65
87,2021-07-30,SPY,438.51


 ---

### Querying for Dates

The following code grabs all the rows of `df_etf` that come after the middle of the month.

In [25]:
df_etf[df_etf['date'] > '2021-07-15']

,date,symbol,close
44,2021-07-16,DIA,346.74
48,2021-07-19,DIA,339.88
52,2021-07-20,DIA,345.08
...,...,...,...
79,2021-07-28,SPY,438.83
83,2021-07-29,SPY,440.65
87,2021-07-30,SPY,438.51


---

**Code Challenge:** Grab all the rows of `df_etf` for the last trade date of the month.

In [26]:
#| code-fold: true
#| code-summary: "Solution"
df_etf[df_etf['date'] == '2021-07-30']

,date,symbol,close
84,2021-07-30,DIA,349.48
85,2021-07-30,IWM,221.05
86,2021-07-30,QQQ,364.57
87,2021-07-30,SPY,438.51


 ---

### Querying on Multiple Criteria

We can filter on muliple criteria by using the `&` operator, which is the vectorized version of `and`.

Suppose that we want all rows for `SPY` that come before July fourth.

In [27]:
bln_ticker = (df_etf['symbol'] == 'SPY')
bln_date = (df_etf['date'] < '2021-07-04')
bln_combined = bln_ticker & bln_date

df_etf[bln_combined]

,date,symbol,close
3,2021-06-30,SPY,428.06
7,2021-07-01,SPY,430.43
11,2021-07-02,SPY,433.72


---

**Code Challenge:** Isolate the rows for `QQQ` and `IWM` on the last trading day before July 4th - try to not use intermediate variables.

In [28]:
df_etf[(df_etf['symbol'].isin(["QQQ", "IWM"])) & (df_etf['date']=='2021-07-02')]

,date,symbol,close
9,2021-07-02,IWM,229.19
10,2021-07-02,QQQ,358.64


--- 

## Querying with `.query()`

I find querying a `DataFrame` via masking to be rather cumbersome.  

I greatly prefer the use of the `DataFrame.query()` method which uses SQL-like strings to define queries.

For example, the following code grabs all the rows corresponding to `IWM`.

In [29]:
df_etf.query('symbol == "IWM"')

,date,symbol,close
1,2021-06-30,IWM,229.37
5,2021-07-01,IWM,231.39
9,2021-07-02,IWM,229.19
...,...,...,...
77,2021-07-28,IWM,220.82
81,2021-07-29,IWM,222.52
85,2021-07-30,IWM,221.05


This code queries all rows corresponding to `QQQ` and `DIA`.

In [30]:
df_etf.query('symbol in ("QQQ", "DIA")')

,date,symbol,close
0,2021-06-30,DIA,344.95
4,2021-07-01,DIA,346.36
8,2021-07-02,DIA,347.94
...,...,...,...
78,2021-07-28,QQQ,365.83
82,2021-07-29,QQQ,366.48
86,2021-07-30,QQQ,364.57


Here we grab the rows corresponding to the first half of July.

In [31]:
df_etf.query('date < "2021-07-15"')

,date,symbol,close
0,2021-06-30,DIA,344.95
4,2021-07-01,DIA,346.36
8,2021-07-02,DIA,347.94
...,...,...,...
31,2021-07-12,SPY,437.08
35,2021-07-13,SPY,435.59
39,2021-07-14,SPY,436.24


And we can filter on multiple criteria via method chaining.  Here we grab all the rows for `SPY` and `IWM` from the second half of the month.

In [32]:
(
df_etf
    .query('symbol in ("SPY", "IWM")')
    .query('date > "2021-07-15"')
)

,date,symbol,close
45,2021-07-16,IWM,214.95
49,2021-07-19,IWM,211.73
53,2021-07-20,IWM,218.30
...,...,...,...
79,2021-07-28,SPY,438.83
83,2021-07-29,SPY,440.65
87,2021-07-30,SPY,438.51


---

**Code Challenge:** Grab all the rows of `df_etf` that correspond to the following criteria:
1. `SPY`
2. first half of month
3. close less than 435

In [33]:
(
df_etf
    .query('symbol == "SPY"')
    .query('date < "2021-07-15"')
    .query('close < 435')
)

,date,symbol,close
3,2021-06-30,SPY,428.06
7,2021-07-01,SPY,430.43
11,2021-07-02,SPY,433.72
15,2021-07-06,SPY,432.93
19,2021-07-07,SPY,434.46
23,2021-07-08,SPY,430.92


---

## Related Reading

*Python Data Science Handbook* - Section 2.6 - Comparisons, Masks, and Boolean Logic

*Python Data Science Handbook* - Section 2.7 - Fancy Indexing

*Python Data Science Handbook* - Section 3.2 - Data Indexing and Selection 

*Python Data Science Handbook* - Section 3.12 - High Performance Pandas